![alt text](../utils/conve.png)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools

import os, sys
sys.path.append(os.path.dirname(os.getcwd()))

from data import WN18

In [2]:
class Config:
    n_epochs = 70
    batch_size = 128
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""

def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    
    triples_all = triples_tr + triples_va + triples_te
    
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}

    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)

    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    s2p2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if s not in s2p2o:
            s2p2o[s] = {p: [o]}
        if p not in s2p2o[s]:
            s2p2o[s][p] = [o]
        if o not in s2p2o[s][p]:
            s2p2o[s][p].append(o)
    return s2p2o


def next_train_batch(triples_tr, e2idx, p2idx, s2p2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i : i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = []
        for (_s, _p, _o) in _triples_tr:
            s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
            temp = np.zeros([len(e2idx)])
            temp[s2p2o[s][p]] = 1.
            y.append(temp)
        y = np.asarray(y)
        yield x_s, x_p, y


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 entity_to_idx, 
                 predicate_to_idx):
    for _i, (s, p, o) in enumerate(eval_triples):
        s_idx, p_idx = entity_to_idx[s], predicate_to_idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               entity_to_idx, 
               predicate_to_idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             entity_to_idx, 
                             predicate_to_idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  entity_to_idx,
                  predicate_to_idx):

    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               entity_to_idx, 
                               predicate_to_idx)))

        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(entity_to_idx)])
        ranks_o, filtered_ranks_o = [], []
    
        for _i, ((s, p, o), scores_o) in enumerate(zip(eval_triples, ScoresO)):
            s_idx, p_idx, o_idx = entity_to_idx[s], predicate_to_idx[p], entity_to_idx[o]

            ranks_o += [1 + np.sum(scores_o > scores_o[o_idx])]
            
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [entity_to_idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            
            filtered_ranks_o += [1 + np.sum(filtered_scores_o > filtered_scores_o[o_idx])]

        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(setting_ranks)
            print('[{}] {} Mean Rank: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [5]:
def alt_concat(x1_x2):
    return tf.concat(x1_x2, 1)


def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.contrib.layers.xavier_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.map_fn(alt_concat, [tf.reshape(s, [10, batch_sz, 20, 1]),
                               tf.reshape(p, [10, batch_sz, 20, 1]),], dtype=tf.float32)
    x = tf.reshape(x, [batch_sz, 20, 20, 1])
    
    x = tf.layers.dropout(x, 0.2, training=is_training)
    x = tf.layers.conv2d(x, 32, (3,3), activation=tf.nn.elu)
    
    x = tf.layers.flatten(x)
    x = tf.layers.dropout(x, 0.5, training=is_training)
    x = tf.layers.dense(x, Config.embed_dim, tf.nn.elu)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        
        global_step = tf.train.get_global_step()
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        lr = {'start': 3e-3, 'end': 8e-4, 'steps': Config.n_epochs * 1100}
        lr_op = tf.train.exponential_decay(lr['start'],
                                           global_step,
                                           lr['steps'],
                                           lr['end']/lr['start'])
        
        train_op = tf.train.AdamOptimizer(lr_op).minimize(loss_op,
                                                          global_step = global_step)
        
        lth = tf.train.LoggingTensorHook({'lr': lr_op,}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op,
                                          training_hooks = [lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))

In [6]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
s2p2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               model_dir = './ConvE',
                               config = tf.estimator.RunConfig(keep_checkpoint_max=1),
                               params = params)

for n_epoch in range(Config.n_epochs):
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, s2p2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using config: {'_model_dir': './ConvE', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1162ba0f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dty

INFO:tensorflow:loss = 2254.8508, step = 2613 (40.894 sec)
INFO:tensorflow:lr = 0.002868461 (40.894 sec)
INFO:tensorflow:global_step/sec: 2.46283
INFO:tensorflow:loss = 1645.2058, step = 2713 (40.604 sec)
INFO:tensorflow:lr = 0.0028635412 (40.603 sec)
INFO:tensorflow:global_step/sec: 2.43861
INFO:tensorflow:loss = 1773.3077, step = 2813 (41.007 sec)
INFO:tensorflow:lr = 0.00285863 (41.008 sec)
INFO:tensorflow:global_step/sec: 2.48947
INFO:tensorflow:loss = 1790.2424, step = 2913 (40.169 sec)
INFO:tensorflow:lr = 0.0028537272 (40.169 sec)
INFO:tensorflow:global_step/sec: 2.51159
INFO:tensorflow:loss = 1613.5806, step = 3013 (39.815 sec)
INFO:tensorflow:lr = 0.0028488326 (39.815 sec)
INFO:tensorflow:global_step/sec: 2.49576
INFO:tensorflow:loss = 1339.8951, step = 3113 (40.069 sec)
INFO:tensorflow:lr = 0.0028439467 (40.069 sec)
INFO:tensorflow:global_step/sec: 2.39696
INFO:tensorflow:loss = 1446.3441, step = 3213 (41.719 sec)
INFO:tensorflow:lr = 0.002839069 (41.719 sec)
INFO:tensorflow:

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-5530
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 5531 into ./ConvE/model.ckpt.
INFO:tensorflow:loss = 633.02167, step = 5531
INFO:tensorflow:lr = 0.0027283204
INFO:tensorflow:global_step/sec: 2.49319
INFO:tensorflow:loss = 582.7348, step = 5631 (40.111 sec)
INFO:tensorflow:lr = 0.0027236412 (40.110 sec)
INFO:tensorflow:global_step/sec: 2.36698
INFO:tensorflow:loss = 524.49225, step = 5731 (42.248 sec)
INFO:tensorflow:lr = 0.0027189697 (42.248 sec)
INFO:tensorflow:global_step/sec: 2.47879
INFO:tensorflow:loss = 437.7215, step = 5831 (40.342 sec)
INFO:tensorflow:lr = 0.0027143066 (40.342 sec)
INFO:tensorflow:global_step/sec: 2.41099
INFO:tensorflow:loss = 549.6943, step = 5931 (41.477 sec)
INFO:tensorflow:lr = 0.0027096514 (41.477 se

INFO:tensorflow:global_step/sec: 2.4166
INFO:tensorflow:loss = 361.2926, step = 8843 (41.380 sec)
INFO:tensorflow:lr = 0.002577535 (41.380 sec)
INFO:tensorflow:Saving checkpoints for 8848 into ./ConvE/model.ckpt.
INFO:tensorflow:Loss for final step: 2.5932164.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-8848
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 875.645
[test] Raw Hits@1: 22.5
[test] Raw Hits@3: 33.42
[test] Raw Hits@5: 38.879999999999995
[test] Raw Hits@10: 45.92
[test] Filtered Mean Rank: 859.6202
[test] Filtered Hits@1: 26.840000000000003
[test] Filtered Hits@3: 37.38
[test] Filtered Hits@5: 42.22
[test] Filtered Hits@10: 49.18
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=flo

INFO:tensorflow:global_step/sec: 2.28157
INFO:tensorflow:loss = 167.3472, step = 11261 (43.829 sec)
INFO:tensorflow:lr = 0.0024727404 (43.830 sec)
INFO:tensorflow:global_step/sec: 2.43614
INFO:tensorflow:loss = 170.1587, step = 11361 (41.049 sec)
INFO:tensorflow:lr = 0.0024684994 (41.049 sec)
INFO:tensorflow:global_step/sec: 2.36342
INFO:tensorflow:loss = 258.04752, step = 11461 (42.311 sec)
INFO:tensorflow:lr = 0.0024642656 (42.311 sec)
INFO:tensorflow:global_step/sec: 2.60736
INFO:tensorflow:loss = 145.16351, step = 11561 (38.353 sec)
INFO:tensorflow:lr = 0.0024600392 (38.353 sec)
INFO:tensorflow:global_step/sec: 2.36576
INFO:tensorflow:loss = 179.62177, step = 11661 (42.270 sec)
INFO:tensorflow:lr = 0.0024558199 (42.270 sec)
INFO:tensorflow:global_step/sec: 2.33404
INFO:tensorflow:loss = 175.24794, step = 11761 (42.844 sec)
INFO:tensorflow:lr = 0.0024516082 (42.844 sec)
INFO:tensorflow:global_step/sec: 2.40882
INFO:tensorflow:loss = 270.34464, step = 11861 (41.514 sec)
INFO:tensorfl

...

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-63042
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 63043 into ./ConvE/model.ckpt.
INFO:tensorflow:loss = 5.588536, step = 63043
INFO:tensorflow:lr = 0.0010165909
INFO:tensorflow:global_step/sec: 2.2783
INFO:tensorflow:loss = 13.39904, step = 63143 (43.894 sec)
INFO:tensorflow:lr = 0.0010148473 (43.893 sec)
INFO:tensorflow:global_step/sec: 2.49056
INFO:tensorflow:loss = 8.058578, step = 63243 (40.152 sec)
INFO:tensorflow:lr = 0.0010131068 (40.152 sec)
INFO:tensorflow:global_step/sec: 2.5172
INFO:tensorflow:loss = 2.3735096, step = 63343 (39.727 sec)
INFO:tensorflow:lr = 0.0010113692 (39.727 sec)
INFO:tensorflow:global_step/sec: 2.33336
INFO:tensorflow:loss = 8.20605, step = 63443 (42.858 sec)
INFO:tensorflow:lr = 0.0010096346 (42.858

INFO:tensorflow:lr = 0.00096205727 (38.858 sec)
INFO:tensorflow:global_step/sec: 2.55598
INFO:tensorflow:loss = 6.5894723, step = 66355 (39.124 sec)
INFO:tensorflow:lr = 0.00096040725 (39.124 sec)
INFO:tensorflow:Saving checkpoints for 66360 into ./ConvE/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00013564377.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./ConvE/model.ckpt-66360
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 527.3104
[test] Raw Hits@1: 31.819999999999997
[test] Raw Hits@3: 45.18
[test] Raw Hits@5: 51.44
[test] Raw Hits@10: 59.28
[test] Filtered Mean Rank: 511.3126
[test] Filtered Hits@1: 39.839999999999996
[test] Filtered Hits@3: 52.94
[test] Filtered Hits@5: 58.52
[test] Filtered Hits@10: 65.48
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=floa

INFO:tensorflow:lr = 0.00092294294 (45.106 sec)
INFO:tensorflow:global_step/sec: 2.26973
INFO:tensorflow:loss = 0.30736417, step = 68773 (44.059 sec)
INFO:tensorflow:lr = 0.00092136 (44.060 sec)
INFO:tensorflow:global_step/sec: 2.2601
INFO:tensorflow:loss = 10.043417, step = 68873 (44.245 sec)
INFO:tensorflow:lr = 0.0009197798 (44.244 sec)
INFO:tensorflow:global_step/sec: 2.3479
INFO:tensorflow:loss = 5.1708994, step = 68973 (42.591 sec)
INFO:tensorflow:lr = 0.0009182022 (42.592 sec)
INFO:tensorflow:global_step/sec: 2.17761
INFO:tensorflow:loss = 6.7426147, step = 69073 (45.922 sec)
INFO:tensorflow:lr = 0.0009166274 (45.922 sec)
INFO:tensorflow:global_step/sec: 2.12548
INFO:tensorflow:loss = 10.70809, step = 69173 (47.049 sec)
INFO:tensorflow:lr = 0.0009150554 (47.048 sec)
INFO:tensorflow:global_step/sec: 2.27009
INFO:tensorflow:loss = 5.6827917, step = 69273 (44.051 sec)
INFO:tensorflow:lr = 0.00091348594 (44.051 sec)
INFO:tensorflow:global_step/sec: 2.20905
INFO:tensorflow:loss = 3.9

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
[test] Raw Mean Rank: 520.0846
[test] Raw Hits@1: 32.28
[test] Raw Hits@3: 45.68
[test] Raw Hits@5: 52.339999999999996
[test] Raw Hits@10: 59.88
[test] Filtered Mean Rank: 504.0634
[test] Filtered Hits@1: 40.42
[test] Filtered Hits@3: 53.64
[test] Filtered Hits@5: 59.199999999999996
[test] Filtered Hits@10: 66.32000000000001
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(200,) dtype=float32_ref>]
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring 

INFO:tensorflow:lr = 0.00083505 (39.300 sec)
INFO:tensorflow:global_step/sec: 2.48834
INFO:tensorflow:loss = 4.9847555, step = 74603 (40.187 sec)
INFO:tensorflow:lr = 0.00083361776 (40.187 sec)
INFO:tensorflow:global_step/sec: 2.43201
INFO:tensorflow:loss = 1.0221622, step = 74703 (41.119 sec)
INFO:tensorflow:lr = 0.00083218806 (41.118 sec)
INFO:tensorflow:global_step/sec: 2.37204
INFO:tensorflow:loss = 4.193805, step = 74803 (42.158 sec)
INFO:tensorflow:lr = 0.0008307608 (42.159 sec)
INFO:tensorflow:global_step/sec: 2.49565
INFO:tensorflow:loss = 3.9946783, step = 74903 (40.069 sec)
INFO:tensorflow:lr = 0.0008293359 (40.069 sec)
INFO:tensorflow:global_step/sec: 2.41961
INFO:tensorflow:loss = 3.7850556, step = 75003 (41.332 sec)
INFO:tensorflow:lr = 0.00082791364 (41.332 sec)
INFO:tensorflow:global_step/sec: 2.40101
INFO:tensorflow:loss = 3.840806, step = 75103 (41.646 sec)
INFO:tensorflow:lr = 0.00082649355 (41.646 sec)
INFO:tensorflow:global_step/sec: 2.43196
INFO:tensorflow:loss = 4